In [1]:
# %pip install peft
# %pip install gradio

import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from peft import PeftModel, PeftConfig 
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, TextStreamer 
import torch 
from threading import Thread 
import gradio as gr 

In [ ]:
peft_model_id = "smangrul/mistral-dpo" # "smangrul/mistral-sft" 
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)
model = PeftModel.from_pretrained(model, peft_model_id)
model.eval()